In [ ]:
import os
os.kill(os.getpid(), 9)


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import fftpack
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import fftpack
from scipy.ndimage import uniform_filter1d
from google.colab.patches import cv2_imshow
from numpy.fft import fft2, ifft2, fftshift
import os

# Make a folder to save videos
os.makedirs("piv_plot", exist_ok=True)
os.makedirs("PIV_PLOT", exist_ok=True)
# os.makedirs("piv_videos1", exist_ok=True)



In [ ]:
import shutil
shutil.rmtree("piv_plot")
shutil.rmtree("PIV_PLOT")

In [ ]:
def extract_frames(cap, saving_fps=10):

    frames = []
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if saving_fps > original_fps:
        saving_fps = original_fps

    frame_interval = int(original_fps / saving_fps)

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frames.append(frame)

        frame_count += 1

    cap.release()
    print(f"Extracted {len(frames)} frames from {total_frames} total frames")

    return frames

In [ ]:
def enhance_contrast(image, method='clahe', alpha=1.5, beta=0):

    if image is None:
        return None

    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    if method == 'simple':
        # Simple linear contrast stretching
        # new_pixel = alpha * gray + beta
        # This enhances the contrast of the grayscale image by scaling (alpha)
        # and shifting (beta) the pixel intensity values.
        enhanced = cv2.addWeighted(gray, alpha, gray, 0, beta)

    elif method == 'clahe':
        # Contrast Limited Adaptive Histogram Equalization
        # Enhances local contrast while avoiding noise amplification
         # clipLimit controls contrast; tileGridSize defines local regions
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(64,64))
        enhanced = clahe.apply(gray)
    elif method == 'equalize':
        # Global Histogram Equalization
        # Redistributes pixel intensities to improve overall image contrast
        enhanced = cv2.equalizeHist(gray)
    elif method == 'adaptive_threshold':
        # Adaptive Thresholding using Mean of local neighborhood
        # Creates a binary image by computing thresholds in small regions
        enhanced = cv2.adaptiveThreshold(gray, 255,
                                         cv2.ADAPTIVE_THRESH_GAUSSIAN_C, # Use Weighted sum of neighborhood pixels
                                         cv2.THRESH_BINARY,          # Output will be binary (black or white)
                                         11, -5)                      # Block size = 11, constant subtracted = 2
    elif method == 'otsu':
        # Otsu's Binarization
        # Automatically calculates the optimal threshold value
        # Good for bimodal histograms (background + foreground)
        _, enhanced = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    elif method == 'combine':
       # CLAHE followed by High-Pass Filtering
       # Step 1: Apply CLAHE to enhance local contrast
      clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(128,128))
      enhanced = clahe.apply(gray)
      # Step 2: Apply a high-pass filter to highlight edges and fine details
      # The kernel enhances areas of rapid intensity change (edges)
      kernel = np.array([[-2,-2,-2], [-2,50,-2], [-2,-2,-2]])
      high_pass = cv2.filter2D(enhanced, -1, kernel)
    elif method == 'binary':
      # Simple Binary Thresholding
      # Pixels above threshold (145) become 255 (white), others become 0 (black
      _, enhanced = cv2.threshold(gray, 145, 255, cv2.THRESH_BINARY)
    else:
            enhanced = gray
    combined = np.hstack((gray, enhanced))
    print(f"real vs enhanced")
    cv2_imshow(combined)
    return enhanced

In [ ]:
def clahe(img, clipLimit=2.0, tileGridSize=(8,8)):
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    clahe_img = clahe.apply(img)
    return clahe_img
    return clahe_img

In [ ]:
def adaptive_threshold(img, blockSize=11, C=2):
    # Apply adaptive thresholding
    adaptive_thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, blockSize, C)
    return adaptive_thresh

In [ ]:
def fft_cross_correlation(win1, win2):
    f1 = fft2(win1)
    f2 = fft2(win2)
    prod = f1 * np.conj(f2)
    cross_corr = fftshift(ifft2(prod).real)
    return cross_corr


def cross_correlation_piv(img1, img2, window_size=64, step=32, pixel_size=0.260,dt=1):
    h, w = img1.shape
    u_field, v_field, positions = [], [], []

    for y in range(0, h - window_size, step):
        for x in range(0, w - window_size, step):
            win1 = img1[y:y+window_size, x:x+window_size]
            win2 = img2[y:y+window_size, x:x+window_size]

            # Normalize (mean subtract)
            win1 = win1 - np.mean(win1)
            win2 = win2 - np.mean(win2)

            # Compute cross-correlation
            corr = fft_cross_correlation(win2, win1)  # win2 is shifted over win1

            # Find peak
            dy, dx = np.unravel_index(np.argmax(corr), corr.shape)

            # Adjust displacement relative to center
            dx -= window_size // 2
            dy -= window_size // 2

            u = dx * pixel_size/dt
            v = dy * pixel_size/dt
            u_field.append(u)
            v_field.append(v)
            positions.append((x + window_size//2, y + window_size//2))

    return np.array(positions), np.array(u_field), np.array(v_field)


In [ ]:
def binned_correlation(U, V, X, Y, delta_r=5.0, use_cosine=True, min_count_threshold=10, smooth=True):
    """
    Compute binned velocity correlation (cosine or dot product) based on pairwise distances.
    Optionally apply smoothing and mask low-count bins.
    """
    n = len(X)
    max_possible_dist = np.sqrt((X.max() - X.min())**2 + (Y.max() - Y.min())**2)
    num_bins = int(np.ceil(max_possible_dist / delta_r)) + 1
    r_bins = (np.arange(num_bins) + 0.5) * delta_r

    corr = np.zeros(num_bins)
    count = np.zeros(num_bins)

    for i in range(n):
        for j in range(i + 1, n):
            dx = X[i] - X[j]
            dy = Y[i] - Y[j]
            r = np.sqrt(dx**2 + dy**2)

            u1, v1 = U[i], V[i]
            u2, v2 = U[j], V[j]

            if use_cosine:
                dot = u1 * u2 + v1 * v2
                mag1 = np.hypot(u1, v1)
                mag2 = np.hypot(u2, v2)
                if mag1 == 0 or mag2 == 0:
                    continue
                value = dot / (mag1 * mag2)
            else:
                value = u1 * u2 + v1 * v2

            bin_index = int(r / delta_r)
            if bin_index < num_bins:
                corr[bin_index] += value
                count[bin_index] += 1

    # Normalize
    with np.errstate(divide='ignore', invalid='ignore'):
        corr = np.where(count > 0, corr / count, 0)

    # Smoothing
    return r_bins, corr



In [ ]:
def binned_correlation_over_time(U_all, V_all, X, Y, delta_r=5.0, use_cosine=True, min_count_threshold=10, smooth=True):
    """
    Compute spatial velocity correlation averaged over time.
    U_all, V_all: shape (T, H, W)
    X, Y: 1D arrays of grid positions (flattened)
    """
    T = U_all.shape[0]
    max_possible_dist = np.sqrt((X.max() - X.min())**2 + (Y.max() - Y.min())**2)
    num_bins = int(np.ceil(max_possible_dist / delta_r)) + 1
    r_bins = (np.arange(num_bins) + 0.5) * delta_r

    corr_total = np.zeros(num_bins)
    count_total = np.zeros(num_bins)

    for t in range(T):
        U = U_all[t].flatten()
        V = V_all[t].flatten()

        # Local correlation per frame with fixed bins
        corr_frame = np.zeros(num_bins)
        count_frame = np.zeros(num_bins)

        for i in range(len(X)):
            for j in range(i + 1, len(X)):
                dx = X[i] - X[j]
                dy = Y[i] - Y[j]
                r = np.sqrt(dx**2 + dy**2)
                bin_index = int(r / delta_r)
                if bin_index >= num_bins:
                    continue

                u1, v1 = U[i], V[i]
                u2, v2 = U[j], V[j]

                if use_cosine:
                    dot = u1 * u2 + v1 * v2
                    mag1 = np.hypot(u1, v1)
                    mag2 = np.hypot(u2, v2)
                    if mag1 == 0 or mag2 == 0:
                        continue
                    value = dot / (mag1 * mag2)
                else:
                    value = u1 * u2 + v1 * v2

                corr_frame[bin_index] += value
                count_frame[bin_index] += 1

        corr_total += corr_frame
        count_total += count_frame

    # Normalize
    with np.errstate(divide='ignore', invalid='ignore'):
        avg_corr = np.where(count_total > 0, corr_total / count_total, 0)

    # Optional: smoothing

    return r_bins, avg_corr


In [ ]:

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
cap = cv2.VideoCapture('/content/low_adhesion.avi')
frames = extract_frames(cap, saving_fps=10)
# def get_video(clip_limit, tile_grid_size, window_step):
def get_video1(blocksize,C,window_step):
  output_frames = []
  output_path =""
  X = np.array([])
  Y = np.array([])
  u = np.array([])
  v = np.array([])
  step_x = window_step[1]
  step_y = window_step[1]

  grid_spacing = np.sqrt(step_x**2 + step_y**2)

  for i in range(len(frames) - 1):
      frame1 = frames[i]
      frame2 = frames[i + 1]
      gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
      gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
      # img1 = clahe(gray1,clipLimit=clip_limit,tileGridSize=tile_grid_size)
      # img2 = clahe(gray2,clipLimit=clip_limit,tileGridSize=tile_grid_size)
      img1 = adaptive_threshold(gray1, blockSize=blocksize, C=C)
      img2 = adaptive_threshold(gray2, blockSize=blocksize, C=C)
      positions, u, v = cross_correlation_piv(img1, img2,window_step[0],window_step[1])
      X, Y = zip(*positions)

      X = np.array(X).reshape(u.shape)
      Y = np.array(Y).reshape(v.shape)
      # print(X.shape,Y.shape)
      N=X.shape[0]
      n=int(N)
      n = int(np.sqrt(N))
      # u and v are 2D numpy arrays
      # mean_magnitude = (np.sum(np.abs(u)) + np.sum(np.abs(v))) / (np.mean(np.abs(u))*np.mean(np.abs(v)))
      U = np.max(np.abs(u))
      V = np.max(np.abs(v))
      fig, ax = plt.subplots(figsize=(8, 8))
      ax.imshow(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))
      start_x = np.reshape(X,(n,n))
      start_y = np.reshape(Y,(n,n))
      start_u = np.reshape(u,(n,n))
      start_v = np.reshape(v,(n,n))

      # if(n//2>10):
      X_cropped = start_x[n//2-n//3:n//2+n//3,n//2-n//3:n//2+n//3]
      Y_cropped = start_y[n//2-n//3:n//2+n//3,n//2-n//3:n//2+n//3]
      u_cropped = start_u[n//2-n//3:n//2+n//3,n//2-n//3:n//2+n//3]
      v_cropped = start_v[n//2-n//3:n//2+n//3,n//2-n//3:n//2+n//3]
      # else:
      #   X_cropped = X
      #   Y_cropped = Y
      #   u_cropped = u
      #   v_cropped = v

      ax.quiver(X_cropped, Y_cropped, u_cropped*window_step[0]/np.sqrt(U**2+V**2) , v_cropped*window_step[1]/V , angles='xy', scale_units='xy', scale=1, color='cyan')
      ax.set_title(f"PIV Frame {i + 1}")
      ax.axis('off')
      canvas = FigureCanvas(fig)
      canvas.draw()
      img_np = np.frombuffer(canvas.buffer_rgba(), dtype=np.uint8)
      img_np = img_np.reshape(canvas.get_width_height()[::-1] + (4,))
      img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGBA2BGR)
      output_frames.append(img_bgr)
      plt.close(fig)

  # Then save inside this folder
  output_path = f"piv_videos1/final_piv_video_{blocksize}_{C}_{window_step[0]}_{window_step[1]}.mp4"

  # Create a VideoWriter object
  height, width = output_frames[0].shape[:2]
  fps = 4  # 12 seconds with 48 frames

  out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
  for frame in output_frames:
      out.write(frame)
  out.release()
  print(f"✅ Your PIV video is saved at: {output_path}")




Extracted 49 frames from 49 total frames


In [ ]:
def get_video(clip_limit, tile_grid_size, window_step):
  output_frames = []
  output_path =""
  step_x = window_step[1]
  step_y = window_step[1]
  save_directory = "piv_plot"
  save_docs = "PIV_PLOT"
  correlation_list = []
  grid_spacing = np.sqrt(step_x**2 + step_y**2)
  base_name = f"{clip_limit}_{tile_grid_size[0]}_{tile_grid_size[1]}_{window_step[0]}_{window_step[1]}"
  for i in range(len(frames) - 1):
      frame1 = frames[i]
      frame2 = frames[i + 1]
      gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
      gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
      img1 = clahe(gray1,clipLimit=clip_limit,tileGridSize=tile_grid_size)
      img2 = clahe(gray2,clipLimit=clip_limit,tileGridSize=tile_grid_size)
      positions, u, v = cross_correlation_piv(img1, img2,window_step[0],window_step[1])
      X, Y = zip(*positions)
      X = np.array(X).reshape(u.shape)
      Y = np.array(Y).reshape(v.shape)
      # print(X.shape,Y.shape)
      N=X.shape[0]
      N=int(N)
      n = int(np.sqrt(N))
      U = np.max(np.abs(u))
      V = np.max(np.abs(v))
      # fig, ax = plt.subplots(figsize=(8, 8))
      # ax.imshow(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))
      start_x = np.reshape(X,(n,n))
      start_y = np.reshape(Y,(n,n))
      start_u = np.reshape(u,(n,n))
      start_v = np.reshape(v,(n,n))
      X_cropped = start_x[n//2-n//3:n//2+n//3,n//2-n//3:n//2+n//3]
      Y_cropped = start_y[n//2-n//3:n//2+n//3,n//2-n//3:n//2+n//3]
      u_cropped = start_u[n//2-n//3:n//2+n//3,n//2-n//3:n//2+n//3]
      v_cropped = start_v[n//2-n//3:n//2+n//3,n//2-n//3:n//2+n//3]
      # # ax.quiver(X_cropped, Y_cropped, u_cropped * 100, v_cropped * 100, angles='xy', scale_units='xy', scale=1, color='cyan')
      # ax.quiver(X_cropped, Y_cropped, u_cropped*window_step[0]/np.sqrt(U**2+V**2) , v_cropped*window_step[1]/V , angles='xy', scale_units='xy', scale=1, color='cyan')
      # # ax.quiver(X, Y, u * 100, -v * 100, angles='xy', scale_units='xy', scale=1, color='cyan')
      # ax.set_title(f"PIV Frame {i + 1}")
      # ax.axis('off')
      # canvas = FigureCanvas(fig)
      # canvas.draw()
      # img_np = np.frombuffer(canvas.buffer_rgba(), dtype=np.uint8)
      # img_np = img_np.reshape(canvas.get_width_height()[::-1] + (4,))
      # img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGBA2BGR)
      # output_frames.append(img_bgr)
      # plt.close(fig)
      # to store all correlation arrays for over-time plot
      # [ ... your CLAHE, cross-correlation, reshaping code ... ]

      # Cropped velocity and position data
      X_cropped = start_x[n//2-n//3:n//2+n//3, n//2-n//3:n//2+n//3]
      Y_cropped = start_y[n//2-n//3:n//2+n//3, n//2-n//3:n//2+n//3]
      u_cropped = start_u[n//2-n//3:n//2+n//3, n//2-n//3:n//2+n//3]
      v_cropped = start_v[n//2-n//3:n//2+n//3, n//2-n//3:n//2+n//3]

      # Flatten for correlation
      U = u_cropped.flatten()
      V = v_cropped.flatten()
      X = X_cropped.flatten()
      Y = Y_cropped.flatten()

      # ➤ PER-FRAME CORRELATION
      r_bins, correlation = binned_correlation(U, V, X, Y, delta_r=5.0, use_cosine=True)
      correlation_list.append(correlation)  # store for later averaging
      # Save per-frame plot
      plt.figure(figsize=(6, 4))
      plt.plot(r_bins, correlation)
      plt.xlabel("Distance r (pixels)")
      plt.ylabel("⟨cos(θ)⟩")
      plt.title(f"Velocity Correlation - Frame {i}")
      plt.grid(True)
      plt.tight_layout()
      plt.savefig(f"{save_directory}/frame_{i:03d}_corr_{base_name}.png")
      plt.close()

# After for loop ends
  if correlation_list:
      correlation_array = np.array(correlation_list)
      avg_corr = np.mean(correlation_array, axis=0)

      # Save average plot
      plt.figure(figsize=(6, 4))
      plt.plot(r_bins, avg_corr, color='red')
      plt.xlabel("Distance r (pixels)")
      plt.ylabel("Mean ⟨cos(θ)⟩ over time")
      plt.title("Time-Averaged Velocity Correlation")
      plt.grid(True)
      plt.tight_layout()
      plt.savefig(f"{save_docs}/average_correlation_{base_name}.png")
      plt.close()

  # # Then save inside this folder
  # output_path = f"piv_videos/final_piv_video_{clip_limit}_{tile_grid_size[0]}_{tile_grid_size[1]}_{window_step[0]}_{window_step[1]}.mp4"

  # # Create a VideoWriter object
  # height, width = output_frames[0].shape[:2]
  # fps = 4  # 12 seconds with 48 frames

  # # out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
  # out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'m', 'p', '4', 'v'), fps, (width, height))

  # for frame in output_frames:
  #     out.write(frame)
  # out.release()
  # print(f"✅ Your PIV video is saved at: {output_path}")

In [ ]:
a=[3.0,4.0]
b=[(16,16),(32,32),(64,64),(128,128)]
c=[(64,32),(128,32),(128,64)]
for i in a:
  for j in b:
    for k in c:
       get_video(i,j,k)

In [ ]:
a=[13,15]
b=[-2,-3,-4,-5]
c=[(64,32),(128,32),(128,64)]
for i in a:
  for j in b:
    for k in c:
       get_video1(i,j,k)

✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-2_64_32.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-2_128_32.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-2_128_64.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-3_64_32.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-3_128_32.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-3_128_64.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-4_64_32.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-4_128_32.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-4_128_64.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-5_64_32.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-5_128_32.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_13_-5_128_64.mp4
✅ Your PIV video is saved at: piv_videos1/final_piv_video_15_-2_64_32.mp4
✅ Your PIV video is saved at: 

In [ ]:
!zip -r piv_plot.zip piv_plot/
!zip -r PIV_PLOT.zip PIV_PLOT/


  adding: piv_plot/ (stored 0%)
  adding: piv_plot/frame_039_corr_4.0_64_64_64_32.png (deflated 3%)
  adding: piv_plot/frame_030_corr_3.0_32_32_128_32.png (deflated 5%)
  adding: piv_plot/frame_033_corr_4.0_64_64_64_32.png (deflated 5%)
  adding: piv_plot/frame_030_corr_3.0_32_32_64_32.png (deflated 5%)
  adding: piv_plot/frame_007_corr_4.0_64_64_64_32.png (deflated 4%)
  adding: piv_plot/frame_030_corr_4.0_16_16_64_32.png (deflated 4%)
  adding: piv_plot/frame_047_corr_3.0_32_32_64_32.png (deflated 4%)
  adding: piv_plot/frame_038_corr_3.0_16_16_128_64.png (deflated 6%)
  adding: piv_plot/frame_041_corr_3.0_32_32_64_32.png (deflated 5%)
  adding: piv_plot/frame_021_corr_4.0_128_128_128_32.png (deflated 3%)
  adding: piv_plot/frame_009_corr_4.0_32_32_64_32.png (deflated 2%)
  adding: piv_plot/frame_011_corr_4.0_64_64_64_32.png (deflated 5%)
  adding: piv_plot/frame_028_corr_3.0_64_64_64_32.png (deflated 3%)
  adding: piv_plot/frame_030_corr_4.0_128_128_64_32.png (deflated 4%)
  adding:

In [ ]:
from google.colab import files
files.download("piv_plot.zip")
files.download("PIV_PLOT.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>